In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pickle
import time
from tqdm.notebook import tqdm

import torch
torch.set_default_tensor_type(torch.DoubleTensor)

from spatial_scene_grammars.nodes import *
from spatial_scene_grammars.rules import *
from spatial_scene_grammars.scene_grammar import *
from spatial_scene_grammars.visualization import *
from spatial_scene_grammars_examples.dish_bin.grammar import *
from spatial_scene_grammars.parsing import *
from spatial_scene_grammars.sampling import *
from spatial_scene_grammars.parameter_estimation import *
from spatial_scene_grammars.dataset import *

import meshcat
import meshcat.geometry as meshcat_geom

Generating from folder  sink/plates_cups_and_bowls/plates
sink:plates_cups_and_bowls:plates:Threshold_Bistro_Ceramic_Dinner_Plate_Ruby_Ring :  <class 'spatial_scene_grammars_examples.dish_bin.grammar.sink:plates_cups_and_bowls:plates:Threshold_Bistro_Ceramic_Dinner_Plate_Ruby_Ring'>
sink:plates_cups_and_bowls:plates:Ecoforms_Plate_S20Avocado :  <class 'spatial_scene_grammars_examples.dish_bin.grammar.sink:plates_cups_and_bowls:plates:Ecoforms_Plate_S20Avocado'>
sink:plates_cups_and_bowls:plates:Room_Essentials_Salad_Plate_Turquoise :  <class 'spatial_scene_grammars_examples.dish_bin.grammar.sink:plates_cups_and_bowls:plates:Room_Essentials_Salad_Plate_Turquoise'>
sink:plates_cups_and_bowls:plates:Ecoforms_Plant_Plate_S11Turquoise :  <class 'spatial_scene_grammars_examples.dish_bin.grammar.sink:plates_cups_and_bowls:plates:Ecoforms_Plant_Plate_S11Turquoise'>
Generating from folder  sink/plates_cups_and_bowls/cups
sink:plates_cups_and_bowls:cups:Cole_Hardware_Mug_Classic_Blue :  <class '

In [2]:
if 'vis' not in globals():
    vis = meshcat.Visualizer()
vis.delete()
base_url = "http://127.0.0.1"
meshcat_url = base_url + ":" + vis.url().split(":")[-1]
print("Meshcat url: ", meshcat_url)
'''
from IPython.display import HTML
HTML("""
    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">
    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>
</div>
""".format(url=meshcat_url))
'''

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Meshcat url:  http://127.0.0.1:7000/static/


'\nfrom IPython.display import HTML\nHTML("""\n    <div style="height: 400px; width: 100%; overflow-x: auto; overflow-y: hidden; resize: both">\n    <iframe src="{url}" style="width: 100%; height: 100%; border: none"></iframe>\n</div>\n""".format(url=meshcat_url))\n'

In [3]:
# Convert dataset to observed node sets (caching output) and draw a few examples.

RECONVERT_DATASET = True
DATASET_YAML_FILE = "sink/saved_scenes.yaml"
DATASET_SAVE_FILE = "observed_node_sets.dat"

if RECONVERT_DATASET or not os.path.exists(DATASET_SAVE_FILE):
    type_map = {
        "bin": DishBin
    }
    model_map = {
    }
    for model_type_set in [PlateModels, CupModels, BowlModels]:
        for model_type in model_type_set:
            # Have to cut off the "sink" folder to match model names;
            # dataset management is ugly and should get reorganized...
            model_map[os.path.join(*model_type.sdf.split("/")[1:])] = model_type
    observed_node_sets = convert_scenes_yaml_to_observed_nodes(DATASET_YAML_FILE, type_map, model_map)
    print("Saving...")
    with open(DATASET_SAVE_FILE, "wb") as f:
        pickle.dump(observed_node_sets, f)

print("Loading...")
with open(DATASET_SAVE_FILE, "rb") as f:
    observed_node_sets = pickle.load(f)

draw_scene_tree_contents_meshcat(
    SceneTree.make_from_observed_nodes(observed_node_sets[0]),
    zmq_url=vis.window.zmq_url, prefix="observed/contents"
)

[2021-09-30 19:02:04.775] [console] [warning] FindResource ignoring DRAKE_RESOURCE_ROOT='/home/gizatt/drake' because it does not contain a 'drake' subdirectory.


Saving...
Loading...


In [4]:
total_of_each_type = {"DishBin": 0}
for model_type_set in [PlateModels, CupModels, BowlModels]:
    for model_type in model_type_set:
        total_of_each_type[model_type.__name__] = 0
for observed_nodes in observed_node_sets:
    for node in observed_nodes:
        total_of_each_type[type(node).__name__] += 1
print(total_of_each_type)

{'DishBin': 30, 'sink:plates_cups_and_bowls:plates:Threshold_Bistro_Ceramic_Dinner_Plate_Ruby_Ring': 12, 'sink:plates_cups_and_bowls:plates:Ecoforms_Plate_S20Avocado': 12, 'sink:plates_cups_and_bowls:plates:Room_Essentials_Salad_Plate_Turquoise': 21, 'sink:plates_cups_and_bowls:plates:Ecoforms_Plant_Plate_S11Turquoise': 0, 'sink:plates_cups_and_bowls:cups:Cole_Hardware_Mug_Classic_Blue': 12, 'sink:plates_cups_and_bowls:cups:Room_Essentials_Mug_White_Yellow': 12, 'sink:plates_cups_and_bowls:cups:Threshold_Porcelain_Coffee_Mug_All_Over_Bead_White': 14, 'sink:plates_cups_and_bowls:bowls:Bradshaw_International_11642_7_Qt_MP_Plastic_Bowl': 9, 'sink:plates_cups_and_bowls:bowls:Cole_Hardware_Bowl_Scirocco_YellowBlue': 10, 'sink:plates_cups_and_bowls:bowls:Room_Essentials_Bowl_Turquiose': 8}


In [5]:
# Draw a random sample from the grammar with its initial params and visualize it.
#torch.random.manual_seed(5)

grammar = SpatialSceneGrammar(
    root_node_type = DishBin,
    root_node_tf = drake_tf_to_torch_tf(RigidTransform(p=[0.5, 0., 0.]))
)

tree = grammar.sample_tree(detach=True)
observed_nodes = tree.get_observed_nodes()

vis["sample"].delete()
draw_scene_tree_contents_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="sample/contents")
draw_scene_tree_structure_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="sample/structure")

In [ ]:
observed_node_set_test = [
    BowlModels[0](drake_tf_to_torch_tf(RigidTransform(
        p=[0.5, 0.0, 0.1],
        R=UniformlyRandomRotationMatrix(RandomGenerator(0))))
    ),
    DishBin(drake_tf_to_torch_tf(
        RigidTransform(p=[0.5, 0., 0.]))
    )
]
print([type(node).__name__ for node in observed_node_set_test])

In [ ]:
supertree = grammar.make_super_tree(max_recursion_depth=10)
print("Supertree size ", len(supertree.nodes))

In [ ]:
# Parse that dummy observation; it should have only one unique parse, since there's only one
# object.
trees = get_optimized_trees_from_mip_results(infer_mle_tree_with_mip(grammar, observed_node_sets[0], N_solutions=5, max_recursion_depth=10, verbose=True))
vis["parses"].delete()
for k, tree in enumerate(trees):
    #draw_scene_tree_contents_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="parses/contents/%d" % k)
    draw_scene_tree_structure_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="parses/structure/%d" % k)

In [ ]:
print(len(trees))

# MIPMAP-EM alternation

In [ ]:
em = EMWrapper(grammar, observed_node_sets)
em.do_iterated_em_fitting(em_iterations=5, tqdm=tqdm, N_solutions=10, num_workers=10, max_recursion_depth=10)

In [ ]:
# Save our grammar params
torch.save(grammar.state_dict(), "fit_grammar.torch")

In [ ]:
em.plot_grammar_parameter_history(DishBin)
em.plot_grammar_parameter_history(Object)
print("Plate")
em.plot_grammar_parameter_history(Plate)
print("PlateStack")
em.plot_grammar_parameter_history(PlateStack)
print("AssortedPlateStacks")
em.plot_grammar_parameter_history(AssortedPlateStacks)
print("AssortedFullBowls")
em.plot_grammar_parameter_history(AssortedFullBowls)

In [ ]:
def get_all_node_xyzs(sampled_trees, node_type):
    l = []
    for tree in sampled_trees:
        for node in tree:
            if isinstance(node, node_type):
                l.append(node.translation.detach().cpu().numpy())
    return np.stack(l)
def plot_post_fit_tree_samples(em, N_samples=100):
    pre_fit_samples= []
    em.grammar.load_state_dict(em.grammar_iters[0])
    for k in range(N_samples):
        pre_fit_samples.append(em.grammar.sample_tree(detach=True))
    fit_samples = []
    em.grammar.load_state_dict(em.grammar_iters[-1])
    for k in range(N_samples):
        fit_samples.append(em.grammar.sample_tree(detach=True))

    for node_type in [PlateModels[0], CupModels[0], BowlModels[0]]:
        plt.figure(dpi=300).set_size_inches(16, 4)
        plt.suptitle("%s XYZ histogram comparisons" % node_type.__name__)

        gt_l = get_all_node_xyzs([SceneTree.make_from_observed_nodes(sample) for sample in observed_node_sets], node_type)
        fit_l = get_all_node_xyzs([sample for sample in fit_samples], node_type)
        pre_fit_l = get_all_node_xyzs([sample for sample in pre_fit_samples], node_type)
        # Pre fit
        for k in range(3):
            plt.subplot(2, 3, k+1)
            plt.title("Before fit %s" % "xyz"[k:(k+1)])
            plt.hist(pre_fit_l[:, k], label="Pre-fitting", alpha=0.5, density=True)
            plt.hist(gt_l[:, k], label="Ground truth", alpha=0.5, density=True)
            if k == 2:
                plt.legend(bbox_to_anchor=(1.1, 1.05))

        for k in range(3):
            plt.subplot(2, 3, k+1+3)
            plt.title("After fit %s" % "xyz"[k:(k+1)])
            plt.hist(fit_l[:, k], label="Post-fitting", alpha=0.5, density=True)
            plt.hist(gt_l[:, k], label="Ground truth", alpha=0.5, density=True)
            if k == 2:
                plt.legend(bbox_to_anchor=(1.1, 1.05))
        plt.tight_layout()
plot_post_fit_tree_samples(em, N_samples=1000)

In [ ]:
# Draw some samples from the fit posterior
vis["fit_samples"].delete()
for k in range(5):
    tree = grammar.sample_tree(detach=True)
    draw_scene_tree_contents_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="fit_samples/%d/contents" % k)
    draw_scene_tree_structure_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="fit_samples/%d/structure" % k)

# Test parsing a scene

In [6]:
# Set up grammar
grammar = SpatialSceneGrammar(
    root_node_type = DishBin,
    root_node_tf = drake_tf_to_torch_tf(RigidTransform(p=[0.5, 0., 0.]))
)
grammar.load_state_dict(torch.load("fit_grammar.torch"))

<All keys matched successfully>

In [9]:
# Parse that dummy observation; it should have only one unique parse, since there's only one
# object.
results = infer_mle_tree_with_mip(grammar, observed_node_sets[0], N_solutions=5, max_recursion_depth=10, verbose=True)
trees = get_optimized_trees_from_mip_results(results)
vis["parses"].delete()
for k, tree in enumerate(trees):
    if k == 0:
        draw_scene_tree_contents_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="parses/contents/%d" % k)
    draw_scene_tree_structure_meshcat(tree, zmq_url=vis.window.zmq_url, prefix="parses/structure/%d" % k)
    print("Computed score %f, optimization score %f" % (tree.score(verbose=0), results.optim_result.get_suboptimal_objective(k)))

Starting setup.
Activation vars allocated.
Continuous variables allocated.
tensor(0.) tensor([0.])
Rule DishBin_37->AssortedFullBowls_7-><spatial_scene_grammars.rules.ProductionRule object at 0x7f6665342f98>, min cost -0.0
tensor(0.) tensor([0.])
Rule DishBin_37->AssortedFullBowls_7-><spatial_scene_grammars.rules.ProductionRule object at 0x7f6665342668>, min cost -0.0
tensor(0.) tensor([0.])
Rule DishBin_37->AssortedFullBowls_7-><spatial_scene_grammars.rules.ProductionRule object at 0x7f6665341048>, min cost -0.0
tensor(0.) tensor([0.])
Rule DishBin_37->AssortedFullBowls_7-><spatial_scene_grammars.rules.ProductionRule object at 0x7f6665341278>, min cost -0.0
tensor(0.) tensor([0.])
Rule DishBin_37->AssortedFullBowls_7-><spatial_scene_grammars.rules.ProductionRule object at 0x7f66653413c8>, min cost -0.0
tensor(5.0169) tensor([-0.0894])
Rule AssortedPlates_7->Plate_44-><spatial_scene_grammars.rules.ProductionRule object at 0x7f6665341160>, min cost -4.927447336145258
tensor(5.0169) tens

tensor(0.) tensor([0.])
Rule Object_18->Cup_37-><spatial_scene_grammars.rules.ProductionRule object at 0x7f66645c6668>, min cost -0.0
tensor(0.) tensor([0.])
Rule Object_18->Cup_37-><spatial_scene_grammars.rules.ProductionRule object at 0x7f66645c6710>, min cost -0.0
tensor(0.) tensor([0.])
Rule Object_18->Cup_37-><spatial_scene_grammars.rules.ProductionRule object at 0x7f66645c67b8>, min cost -0.0
tensor(0.) tensor([0.])
Rule Object_19->Cup_38-><spatial_scene_grammars.rules.ProductionRule object at 0x7f66645c68d0>, min cost -0.0
tensor(0.) tensor([0.])
Rule Object_19->Cup_38-><spatial_scene_grammars.rules.ProductionRule object at 0x7f66645c6978>, min cost -0.0
tensor(0.) tensor([0.])
Rule Object_19->Cup_38-><spatial_scene_grammars.rules.ProductionRule object at 0x7f66645c6a20>, min cost -0.0
tensor(0.) tensor([0.])
Rule Object_20->Cup_39-><spatial_scene_grammars.rules.ProductionRule object at 0x7f66645c6c88>, min cost -0.0
tensor(0.) tensor([0.])
Rule Object_20->Cup_39-><spatial_scene

Optimization success?:  True
Logfile: 

Gurobi 9.0.2 (linux64) logging started Thu Sep 30 19:07:49 2021

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 4232 rows, 2985 columns and 17712 nonzeros
Model fingerprint: 0x0c7c47a8
Model has 213 quadratic objective terms
Variable types: 2724 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-13, 3e+04]
  QObjective range [3e+01, 2e+06]
  Bounds range     [1e-01, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 3393 rows and 2299 columns
Presolve time: 0.03s
Presolved: 839 rows, 686 columns, 3605 nonzeros
Presolved model has 132 quadratic objective terms
Variable types: 672 continuous, 14 integer (14 binary)

Root relaxation: objective -2.181082e+03, 1454 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

Rot:  0.0
Object_17:Plate_49: child 0.000000
XYZ:  0.0
Rot:  0.0
Object_17:Bowl_42: child 0.000000
XYZ:  0.0
Rot:  0.0
Object_17:Cup_36: child 0.000000
Object_18: child set score -inf
XYZ:  0.0
Rot:  0.0
Object_18:Plate_50: child 0.000000
XYZ:  0.0
Rot:  0.0
Object_18:Bowl_43: child 0.000000
XYZ:  0.0
Rot:  0.0
Object_18:Cup_37: child 0.000000
Object_19: child set score -inf
XYZ:  0.0
Rot:  0.0
Object_19:Plate_51: child 0.000000
XYZ:  0.0
Rot:  0.0
Object_19:Bowl_44: child 0.000000
XYZ:  0.0
Rot:  0.0
Object_19:Cup_38: child 0.000000
sink:plates_cups_and_bowls:bowls:Bradshaw_International_11642_7_Qt_MP_Plastic_Bowl_46: child set score 0.000000
sink:plates_cups_and_bowls:bowls:Cole_Hardware_Bowl_Scirocco_YellowBlue_48: child set score 0.000000
sink:plates_cups_and_bowls:bowls:Room_Essentials_Bowl_Turquiose_46: child set score 0.000000
Object_20: child set score -inf
XYZ:  0.0
Rot:  0.0
Object_20:Plate_52: child 0.000000
XYZ:  0.0
Rot:  0.0
Object_20:Bowl_45: child 0.000000
XYZ:  0.0
Rot

XYZ:  0.0
Rot:  0.0
Cup_34:sink:plates_cups_and_bowls:cups:Room_Essentials_Mug_White_Yellow_43: child 0.000000
XYZ:  0.0
Rot:  0.0
Cup_34:sink:plates_cups_and_bowls:cups:Threshold_Porcelain_Coffee_Mug_All_Over_Bead_White_44: child 0.000000
Cup_35: child set score -inf
XYZ:  0.0
Rot:  0.0
Cup_35:sink:plates_cups_and_bowls:cups:Cole_Hardware_Mug_Classic_Blue_44: child 0.000000
XYZ:  0.0
Rot:  0.0
Cup_35:sink:plates_cups_and_bowls:cups:Room_Essentials_Mug_White_Yellow_44: child 0.000000
XYZ:  0.0
Rot:  0.0
Cup_35:sink:plates_cups_and_bowls:cups:Threshold_Porcelain_Coffee_Mug_All_Over_Bead_White_45: child 0.000000
Bowl_34: child set score -1.098612
XYZ:  0.0
Rot:  0.0
Bowl_34:sink:plates_cups_and_bowls:bowls:Bradshaw_International_11642_7_Qt_MP_Plastic_Bowl_39: child 0.000000
XYZ:  0.0
Rot:  0.0
Bowl_34:sink:plates_cups_and_bowls:bowls:Cole_Hardware_Bowl_Scirocco_YellowBlue_41: child 0.000000
XYZ:  0.0
Rot:  0.0
Bowl_34:sink:plates_cups_and_bowls:bowls:Room_Essentials_Bowl_Turquiose_39: ch

...

Rot:  -2.3101159788857566
BowlContents_10:Object_22: child -106.282588
sink:plates_cups_and_bowls:plates:Threshold_Bistro_Ceramic_Dinner_Plate_Ruby_Ring_52: child set score 0.000000
sink:plates_cups_and_bowls:plates:Ecoforms_Plate_S20Avocado_54: child set score 0.000000
sink:plates_cups_and_bowls:plates:Room_Essentials_Salad_Plate_Turquoise_61: child set score 0.000000
sink:plates_cups_and_bowls:plates:Ecoforms_Plant_Plate_S11Turquoise_41: child set score 0.000000
sink:plates_cups_and_bowls:plates:Threshold_Bistro_Ceramic_Dinner_Plate_Ruby_Ring_53: child set score 0.000000
sink:plates_cups_and_bowls:plates:Ecoforms_Plate_S20Avocado_55: child set score 0.000000
sink:plates_cups_and_bowls:plates:Room_Essentials_Salad_Plate_Turquoise_62: child set score 0.000000
sink:plates_cups_and_bowls:plates:Ecoforms_Plant_Plate_S11Turquoise_42: child set score 0.000000
sink:plates_cups_and_bowls:plates:Threshold_Bistro_Ceramic_Dinner_Plate_Ruby_Ring_54: child set score 0.000000
sink:plates_cups_and_b

XYZ:  0.0
Rot:  0.0
Cup_30:sink:plates_cups_and_bowls:cups:Cole_Hardware_Mug_Classic_Blue_39: child 0.000000
XYZ:  0.0
Rot:  0.0
Cup_30:sink:plates_cups_and_bowls:cups:Room_Essentials_Mug_White_Yellow_39: child 0.000000
XYZ:  0.0
Rot:  0.0
Cup_30:sink:plates_cups_and_bowls:cups:Threshold_Porcelain_Coffee_Mug_All_Over_Bead_White_40: child 0.000000
Cup_31: child set score -inf
XYZ:  0.0
Rot:  0.0
Cup_31:sink:plates_cups_and_bowls:cups:Cole_Hardware_Mug_Classic_Blue_40: child 0.000000
XYZ:  0.0
Rot:  0.0
Cup_31:sink:plates_cups_and_bowls:cups:Room_Essentials_Mug_White_Yellow_40: child 0.000000
XYZ:  0.0
Rot:  0.0
Cup_31:sink:plates_cups_and_bowls:cups:Threshold_Porcelain_Coffee_Mug_All_Over_Bead_White_41: child 0.000000
Cup_32: child set score -inf
XYZ:  0.0
Rot:  0.0
Cup_32:sink:plates_cups_and_bowls:cups:Cole_Hardware_Mug_Classic_Blue_41: child 0.000000
XYZ:  0.0
Rot:  0.0
Cup_32:sink:plates_cups_and_bowls:cups:Room_Essentials_Mug_White_Yellow_41: child 0.000000
XYZ:  0.0
Rot:  0.0
Cup_

XYZ:  0.0
Rot:  0.0
Bowl_46:sink:plates_cups_and_bowls:bowls:Room_Essentials_Bowl_Turquiose_51: child 0.000000
Cup_40: child set score -inf
XYZ:  0.0
Rot:  0.0
Cup_40:sink:plates_cups_and_bowls:cups:Cole_Hardware_Mug_Classic_Blue_49: child 0.000000
XYZ:  0.0
Rot:  0.0
Cup_40:sink:plates_cups_and_bowls:cups:Room_Essentials_Mug_White_Yellow_49: child 0.000000
XYZ:  0.0
Rot:  0.0
Cup_40:sink:plates_cups_and_bowls:cups:Threshold_Porcelain_Coffee_Mug_All_Over_Bead_White_50: child 0.000000
Plate_54: child set score -inf
XYZ:  0.0
Rot:  0.0
Plate_54:sink:plates_cups_and_bowls:plates:Threshold_Bistro_Ceramic_Dinner_Plate_Ruby_Ring_61: child 0.000000
XYZ:  0.0
Rot:  0.0
Plate_54:sink:plates_cups_and_bowls:plates:Ecoforms_Plate_S20Avocado_63: child 0.000000
XYZ:  0.0
Rot:  0.0
Plate_54:sink:plates_cups_and_bowls:plates:Room_Essentials_Salad_Plate_Turquoise_70: child 0.000000
XYZ:  0.0
Rot:  0.0
Plate_54:sink:plates_cups_and_bowls:plates:Ecoforms_Plant_Plate_S11Turquoise_50: child 0.000000
Bowl_4

Plate_48:sink:plates_cups_and_bowls:plates:Ecoforms_Plant_Plate_S11Turquoise_44: child 0.000000
Bowl_40: child set score -1.098612
XYZ:  0.0
Rot:  0.0
Bowl_40:sink:plates_cups_and_bowls:bowls:Bradshaw_International_11642_7_Qt_MP_Plastic_Bowl_45: child 0.000000
XYZ:  0.0
Rot:  0.0
Bowl_40:sink:plates_cups_and_bowls:bowls:Cole_Hardware_Bowl_Scirocco_YellowBlue_47: child 0.000000
XYZ:  0.0
Rot:  0.0
Bowl_40:sink:plates_cups_and_bowls:bowls:Room_Essentials_Bowl_Turquiose_45: child 0.000000
BowlContents_9: child set score -0.126800
XYZ:  -103.0835869221151
Rot:  -3.2409834815264165
BowlContents_9:Object_17: child -106.324570
XYZ:  -190.83419486055183
Rot:  -2.310083461403838
BowlContents_9:Object_18: child -193.144278
XYZ:  -190.83419486055183
Rot:  -2.3101159788857593
BowlContents_9:Object_19: child -193.144311
Bowl_41: child set score -inf
XYZ:  0.0
Rot:  0.0
Bowl_41:sink:plates_cups_and_bowls:bowls:Bradshaw_International_11642_7_Qt_MP_Plastic_Bowl_46: child 0.000000
XYZ:  0.0
Rot:  0.0
B

Computed score -7.598478, optimization score -2174.877598
Computed score 15.222591, optimization score -1685.371706
Computed score 15.222591, optimization score -1685.371706
Computed score -672.038982, optimization score -1205.034282
Computed score -674.330799, optimization score -1202.742520
